# Análise de Jogos da NFL com Regressão Logística

Este notebook utiliza a regressão logística para prever o vencedor de jogos da NFL com base em vitórias e derrotas acumuladas dos times. A seguir, apresentamos o passo a passo do código:

---

## 1. Importação de Bibliotecas

Importamos as bibliotecas necessárias para manipulação de dados, divisão de conjuntos, criação do modelo e avaliação de desempenho.

## 2. Carregamento do dataset

Utilizei o datset "games.csv", que contém dados de jogos da NFL de 1999 até os jogos atuais, o dataset é atualizado toda semana e eu atualizo através desse repositório: "https://github.com/nflverse/nfldata/blob/master/data/games.csv"

## 3. Filtragem dos dados
Filtrei apenas os jogos de temporada regular, de 2013 pra frente

## 4. Criação de features

Vitória e derrotas acumuladas no time de casa e fora

## 5. Divisão em treino e teste

Utilizei 80% dos dados para treino e 20% para teste

## 6. Treinamento do modelo

Criei um moelo de regressão logística, que visa predizer o vencedor da partida

## 7. Avaliação do modelo

Calcula a acurácia do modelo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from rich.console import Console
from rich.table import Table

In [2]:
# Carregar dataset
df = pd.read_csv('games.csv')

# Filtrar temporadas a partir de 2013 e jogos da temporada regular
df = df[df['season'] >= 2013]
df = df[df['game_type'] == 'REG']

# Definir o vencedor (1 se o time da casa vencer, 0 se o time visitante vencer)
df['winner'] = df['result'].apply(lambda x: 1 if x > 0 else 0)
df = df[df['result'] != 0]

# Contar vitórias do time da casa
df['home_team_wins'] = df.groupby(['home_team', 'season'])['winner'].cumsum()

# Contar vitórias do time visitante (inverso das vitórias do time da casa)
df['away_team_wins'] = df.groupby(['away_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time da casa (inverso das vitórias do time da casa)
df['home_team_losses'] = df.groupby(['home_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time visitante
df['away_team_losses'] = df.groupby(['away_team', 'season'])['winner'].cumsum()

# Selecionar as colunas de interesse para o modelo
parametros = ['home_team_wins', 'away_team_wins', 'home_team_losses', 'away_team_losses']

# Dividir em treino e teste
df_train = df[df['season'] < 2023]
df_test = df[df['season'] >= 2023]

X_train = df_train[parametros]
y_train = df_train['winner']
X_test = df_test[parametros]
y_test = df_test['winner']

# Criar o modelo de regressão logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Fazer previsões
y_pred = modelo.predict(X_test)

# Avaliar a acurácia do modelo
acuracia = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acuracia * 100:.2f}%')

Acurácia: 83.64%


In [3]:
df

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,away_coach,home_coach,referee,stadium_id,stadium,winner,home_team_wins,away_team_wins,home_team_losses,away_team_losses
3714,2013_01_BAL_DEN,2013,REG,1,2013-09-05,Thursday,20:30,BAL,27.0,DEN,...,John Harbaugh,John Fox,Walt Coleman,DEN00,Sports Authority Field at Mile High,1,1,0,0,1
3715,2013_01_NE_BUF,2013,REG,1,2013-09-08,Sunday,13:00,NE,23.0,BUF,...,Bill Belichick,Doug Marrone,Walt Anderson,BUF00,Ralph Wilson Stadium,0,0,1,1,0
3716,2013_01_SEA_CAR,2013,REG,1,2013-09-08,Sunday,13:00,SEA,12.0,CAR,...,Pete Carroll,Ron Rivera,Jeff Triplette,CAR00,Bank of America Stadium,0,0,1,1,0
3717,2013_01_CIN_CHI,2013,REG,1,2013-09-08,Sunday,13:00,CIN,21.0,CHI,...,Marvin Lewis,Marc Trestman,Carl Cheffers,CHI98,Soldier Field,1,1,0,0,1
3718,2013_01_MIA_CLE,2013,REG,1,2013-09-08,Sunday,13:00,MIA,23.0,CLE,...,Joe Philbin,Rob Chudzinski,Mike Carey,CLE00,FirstEnergy Stadium,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973,2024_18_MIA_NYJ,2024,REG,18,2025-01-05,Sunday,13:00,MIA,NaN,NYJ,...,Mike McDaniel,Robert Saleh,NaN,NYC01,MetLife Stadium,0,2,4,6,5
6974,2024_18_NYG_PHI,2024,REG,18,2025-01-05,Sunday,13:00,NYG,NaN,PHI,...,Brian Daboll,Nick Sirianni,NaN,PHI00,Lincoln Financial Field,0,6,3,3,5
6975,2024_18_CIN_PIT,2024,REG,18,2025-01-05,Sunday,13:00,CIN,NaN,PIT,...,Zac Taylor,Mike Tomlin,NaN,PIT00,Acrisure Stadium,0,5,6,3,3
6976,2024_18_NO_TB,2024,REG,18,2025-01-05,Sunday,13:00,NO,NaN,TB,...,Dennis Allen,Todd Bowles,NaN,TAM00,Raymond James Stadium,0,3,4,6,4


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Função para calcular o lucro de uma aposta
def calcular_lucro(row, valor_entrada):
    if row['predicted_winner'] == 1:  # Aposta no time da casa
        odds = row['home_moneyline']
        if row['winner'] == 1:  # Aposta certa (time da casa venceu)
            return valor_entrada * (odds / 100) if odds > 0 else valor_entrada * (100 / abs(odds))
        else:  # Aposta errada (time da casa perdeu)
            return -valor_entrada
    elif row['predicted_winner'] == 0:  # Aposta no time visitante
        odds = row['away_moneyline']
        if row['winner'] == 0:  # Aposta certa (time visitante venceu)
            return valor_entrada * (odds / 100) if odds > 0 else valor_entrada * (100 / abs(odds))
        else:  # Aposta errada (time visitante perdeu)
            return -valor_entrada
    else:  # Caso não haja previsão
        return 0

# Função para realizar o backtest respeitando a cronologia
def realizar_backtest(df, parametros, modelo, valor_entrada=100):
    # Ordenar os dados cronologicamente
    df = df.sort_values(by=['season', 'gameday'])
    
    resultados = []
    rois_por_temporada = []
    
    # Iterar ano a ano para simular um cenário real de predição
    for temporada in sorted(df['season'].unique()):
        # Dividir em treino (jogos anteriores à temporada atual) e teste (jogos da temporada atual)
        df_treino = df[df['season'] < temporada]
        df_teste = df[df['season'] == temporada]
        
        # Verificar se há dados suficientes para treinar
        if len(df_treino) == 0 or len(df_teste) == 0:
            continue
        
        X_treino = df_treino[parametros]
        y_treino = df_treino['winner']
        X_teste = df_teste[parametros]
        y_teste = df_teste['winner']
        
        # Treinar o modelo apenas com os dados anteriores
        modelo.fit(X_treino, y_treino)
        
        # Fazer previsões para a temporada atual
        y_pred = modelo.predict(X_teste)
        df_teste['predicted_winner'] = y_pred
        
        # Calcular a acurácia para a temporada
        acuracia_temporada = accuracy_score(y_teste, y_pred)
        resultados.append({'temporada': temporada, 'acuracia': acuracia_temporada})
        print(f'Acurácia para a temporada {temporada}: {acuracia_temporada * 100:.2f}%')
        
        # Calcular o ROI para a temporada
        df_teste['lucro'] = df_teste.apply(calcular_lucro, axis=1, valor_entrada=valor_entrada)
        lucro_total = df_teste['lucro'].sum()
        valor_investido_total = len(df_teste) * valor_entrada
        roi = (lucro_total / valor_investido_total) * 100  # ROI em percentual
        rois_por_temporada.append(roi)
        print(f"ROI para a temporada {temporada}: {roi:.2f}%")
    
    # Calcular a acurácia média do backtest
    acuracia_media = sum([r['acuracia'] for r in resultados]) / len(resultados)
    print(f'Acurácia média no backtest: {acuracia_media * 100:.2f}%')
    
    # Calcular o ROI médio do backtest
    roi_medio = sum(rois_por_temporada) / len(rois_por_temporada)
    print(f"ROI médio no backtest: {roi_medio:.2f}%")
    
    return resultados, rois_por_temporada

# Configurar o modelo
modelo = LogisticRegression()

# Realizar o backtest
resultados_backtest, rois_backtest = realizar_backtest(df, parametros, modelo)


Acurácia para a temporada 2014: 86.27%
ROI para a temporada 2014: 55.34%
Acurácia para a temporada 2015: 80.86%
ROI para a temporada 2015: 50.61%
Acurácia para a temporada 2016: 86.22%
ROI para a temporada 2016: 59.64%
Acurácia para a temporada 2017: 81.64%
ROI para a temporada 2017: 41.76%
Acurácia para a temporada 2018: 85.04%
ROI para a temporada 2018: 49.64%
Acurácia para a temporada 2019: 83.53%
ROI para a temporada 2019: 50.19%
Acurácia para a temporada 2020: 86.67%
ROI para a temporada 2020: 50.65%
Acurácia para a temporada 2021: 82.29%
ROI para a temporada 2021: 47.30%
Acurácia para a temporada 2022: 82.90%
ROI para a temporada 2022: 46.54%
Acurácia para a temporada 2023: 81.99%
ROI para a temporada 2023: 44.73%
Acurácia para a temporada 2024: 85.29%
ROI para a temporada 2024: 48.24%
Acurácia média no backtest: 83.88%
ROI médio no backtest: 49.51%


C:\Users\guepp\AppData\Local\Temp\ipykernel_7644\418479454.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste['predicted_winner'] = y_pred
C:\Users\guepp\AppData\Local\Temp\ipykernel_7644\418479454.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste['lucro'] = df_teste.apply(calcular_lucro, axis=1, valor_entrada=valor_entrada)
C:\Users\guepp\AppData\Local\Temp\ipykernel_7644\418479454.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [5]:
import os
from rich.console import Console
from rich.table import Table

# Instanciar o console do Rich
console = Console()

# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Fazer previsões de probabilidade
y_proba_semana = modelo.predict_proba(df_semana[parametros])

# Adicionar as colunas de probabilidade no DataFrame
df_semana['home_team_win_prob'] = y_proba_semana[:, 1]  # Probabilidade do time da casa vencer
df_semana['away_team_win_prob'] = y_proba_semana[:, 0]  # Probabilidade do time visitante vencer

# Adicionar a coluna de vencedor previsto
df_semana['predicted_winner'] = modelo.predict(df_semana[parametros])

# Caminho do arquivo CSV
arquivo_csv = f"previsões/Previsões_semana_{semana}.csv"

# Verificar se o arquivo já existe
if os.path.exists(arquivo_csv):
    console.print(f"[bold yellow]O arquivo '{arquivo_csv}' já existe.[/bold yellow]")
    resposta = input("Deseja sobrescrever? (s/n): ").strip().lower()
    if resposta != 's':
        console.print("[bold red]Operação cancelada. O arquivo não foi sobrescrito.[/bold red]")
    else:
        df_semana.to_csv(arquivo_csv, index=False)
        console.print(f"[bold green]As previsões foram sobrescritas no arquivo '{arquivo_csv}'.[/bold green]")
else:
    df_semana.to_csv(arquivo_csv, index=False)
    console.print(f"[bold green]As previsões foram salvas no arquivo '{arquivo_csv}'.[/bold green]")

# Criar a tabela do Rich
table = Table(title=f"Predições para os Jogos da Semana {semana}")

# Adicionar colunas na tabela
table.add_column("Time da Casa", style="bold cyan")
table.add_column("Time Visitante", style="bold cyan")
table.add_column("Chance Casa (%)", style="green")
table.add_column("Chance Visitante (%)", style="red")
table.add_column("Vencedor Previsto", style="bold magenta")

# Adicionar as linhas de predições
for _, row in df_semana.iterrows():
    vencedor = row['home_team'] if row['predicted_winner'] == 1 else row['away_team']
    table.add_row(
        row['home_team'],
        row['away_team'],
        f"{row['home_team_win_prob']:.2%}",
        f"{row['away_team_win_prob']:.2%}",
        vencedor
    )

# Exibir a tabela
console.print(table)


Digite o número da semana (por exemplo, 1 para semana 1):  1


O arquivo 'previsões/Previsões_semana_1.csv' já existe.

Deseja sobrescrever? (s/n):  s


As previsões foram sobrescritas no arquivo 'previsões/Previsões_semana_1.csv'.

                             Predições para os Jogos da Semana 1                              
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Time da Casa ┃ Time Visitante ┃ Chance Casa (%) ┃ Chance Visitante (%) ┃ Vencedor Previsto ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ KC           │ BAL            │ 81.45%          │ 18.55%               │ KC                │
│ PHI          │ GB             │ 81.45%          │ 18.55%               │ PHI               │
│ ATL          │ PIT            │ 23.58%          │ 76.42%               │ PIT               │
│ BUF          │ ARI            │ 81.45%          │ 18.55%               │ BUF               │
│ CHI          │ TEN            │ 81.45%          │ 18.55%               │ CHI               │
│ CIN          │ NE             │ 23.58%          │ 76.42%               │ NE                │
│ IND          │ HOU            │ 23.58%          │ 76.42%               │ HOU               │
│ MIA          │ JAX            │ 81.45%          │ 18.55%               │ MIA               │
│ NO           │ CAR            │ 81.45%          │ 18.55%               │ NO                │
│ NYG          │ MIN            │ 23.58%          │ 76.42%               │ MIN               │
│ LAC          │ LV             │ 81.45%          │ 18.55%               │ LAC               │
│ SEA          │ DEN            │ 81.45%          │ 18.55%               │ SEA               │
│ CLE          │ DAL            │ 23.58%          │ 76.42%               │ DAL               │
│ TB           │ WAS            │ 81.45%          │ 18.55%               │ TB                │
│ DET          │ LA             │ 81.45%          │ 18.55%               │ DET               │
│ SF           │ NYJ            │ 81.45%          │ 18.55%               │ SF                │
└──────────────┴────────────────┴─────────────────┴──────────────────────┴───────────────────┘

In [6]:
def calcular_lucro(row, valor_entrada):
    if row['predicted_winner'] == 1:  # Aposta no time da casa
        odds = row['home_moneyline']
        if row['winner'] == 1:  # Aposta certa (time da casa venceu)
            return valor_entrada * (odds / 100) if odds > 0 else valor_entrada * (100 / abs(odds))
        else:  # Aposta errada (time da casa perdeu)
            return -valor_entrada
    elif row['predicted_winner'] == 0:  # Aposta no time visitante
        odds = row['away_moneyline']
        if row['winner'] == 0:  # Aposta certa (time visitante venceu)
            return valor_entrada * (odds / 100) if odds > 0 else valor_entrada * (100 / abs(odds))
        else:  # Aposta errada (time visitante perdeu)
            return -valor_entrada
    else:  # Caso não haja previsão
        return 0

In [7]:
def moneyline_to_decimal(moneyline):
    if moneyline > 0:
        return (moneyline / 100) + 1
    elif moneyline < 0:
        return (100 / abs(moneyline)) + 1
    else:
        return None  # Odds inválidas

In [8]:
from rich.console import Console
from rich.table import Table# Instanciar o console do Rich
console = Console()

# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Perguntar o valor da entrada nos jogos
valor_entrada = float(input("Digite o valor da entrada para cada jogo (exemplo: 100): "))

# Carregar as previsões para a semana escolhida
previsao = pd.read_csv(f'previsões/Previsões_semana_{semana}.csv')

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Converter odds Moneyline para Decimais
df_semana['home_odds_decimal'] = df_semana['home_moneyline'].apply(moneyline_to_decimal)
df_semana['away_odds_decimal'] = df_semana['away_moneyline'].apply(moneyline_to_decimal)

# Adicionar coluna de previsão de vencedor
if 'predicted_winner' not in df_semana.columns:
    df_semana['predicted_winner'] = "Sem previsão"  # Inicializar a coluna, se não existir
else:
    df_semana['predicted_winner'] = df_semana['predicted_winner'].fillna("Sem previsão")

# Atribuir valores da previsão de acordo com o CSV
df_semana['predicted_winner'] = df_semana['game_id'].map(
    previsao.set_index('game_id')['predicted_winner']
)

# Calcular o lucro para cada jogo
df_semana['lucro'] = df_semana.apply(calcular_lucro, axis=1, valor_entrada=valor_entrada)

# Criar a tabela do Rich
table = Table(title=f"Comparação de Previsões para os Jogos da Semana {semana}")

# Adicionar colunas na tabela
table.add_column("Time da Casa", style="bold cyan")
table.add_column("Time Visitante", style="bold cyan")
table.add_column("Vencedor Previsto", style="bold magenta")
table.add_column("Vencedor Real", style="yellow")
table.add_column("Odds Casa", style="green")
table.add_column("Odds Visitante", style="red")
table.add_column("Lucro", style="bold green")

# Adicionar as linhas com os dados para cada jogo
for _, row in df_semana.iterrows():
    predicted_team = row['home_team'] if row['predicted_winner'] == 1 else (
        row['away_team'] if row['predicted_winner'] == 0 else "Sem previsão"
    )
    real_winner_team = (
        row['home_team'] if row['winner'] == 1 
        else row['away_team'] if row['winner'] == 0 
        else "Ainda não definido"
    )
    lucro = row['lucro'] if 'lucro' in row else 0
    
    table.add_row(
        row['home_team'],
        row['away_team'],
        predicted_team,
        real_winner_team,
        f"{row['home_odds_decimal']:.2f}",
        f"{row['away_odds_decimal']:.2f}",
        f"${lucro:.2f}"
    )

# Exibir a tabela
console.print(table)

# Calcular métricas da semana
acertos = (df_semana['predicted_winner'] == df_semana['winner']).sum()
total_jogos = len(df_semana[df_semana['winner'].notna()])  # Ignorar jogos sem resultado
assertividade = (acertos / total_jogos * 100) if total_jogos > 0 else 0
lucro_total = df_semana['lucro'].sum() if 'lucro' in df_semana else 0
porcentagem_lucro = (lucro_total / (total_jogos * valor_entrada)) * 100 if total_jogos > 0 else 0

# Exibir resultado geral da semana
result_table = Table(title="Resultados da Semana")
result_table.add_column("Métrica", style="bold cyan")
result_table.add_column("Valor", style="bold magenta")

result_table.add_row("Total de jogos", str(total_jogos))
result_table.add_row("Acertos", str(acertos))
result_table.add_row("Assertividade", f"{assertividade:.2f}%")
result_table.add_row("Lucro total", f"${lucro_total:.2f}")
result_table.add_row("Porcentagem de lucro", f"{porcentagem_lucro:.2f}%")

console.print(result_table)

Digite o número da semana (por exemplo, 1 para semana 1):  16
Digite o valor da entrada para cada jogo (exemplo: 100):  5


                            Comparação de Previsões para os Jogos da Semana 16                             
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Time da Casa ┃ Time Visitante ┃ Vencedor Previsto ┃ Vencedor Real ┃ Odds Casa ┃ Odds Visitante ┃ Lucro  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ LAC          │ DEN            │ DEN               │ LAC           │ 1.68      │ 2.24           │ $-5.00 │
│ KC           │ HOU            │ KC                │ KC            │ 1.56      │ 2.50           │ $2.78  │
│ BAL          │ PIT            │ PIT               │ BAL           │ 1.29      │ 3.75           │ $-5.00 │
│ ATL          │ NYG            │ NYG               │ ATL           │ 1.18      │ 5.10           │ $-5.00 │
│ CAR          │ ARI            │ ARI               │ CAR           │ 3.00      │ 1.41           │ $-5.00 │
│ CHI          │ DET            │ DET               │ DET           │ 3.70      │ 1.29           │ $1.47  │
│ CIN          │ CLE            │ CLE               │ CIN           │ 1.17      │ 5.25           │ $-5.00 │
│ IND          │ TEN            │ IND               │ IND           │ 1.51      │ 2.64           │ $2.53  │
│ NYJ          │ LA             │ LA                │ LA            │ 2.36      │ 1.62           │ $3.09  │
│ WAS          │ PHI            │ PHI               │ WAS           │ 2.80      │ 1.46           │ $-5.00 │
│ SEA          │ MIN            │ MIN               │ MIN           │ 2.24      │ 1.68           │ $3.38  │
│ BUF          │ NE             │ BUF               │ BUF           │ 1.09      │ 8.00           │ $0.45  │
│ LV           │ JAX            │ JAX               │ LV            │ 1.77      │ 2.10           │ $-5.00 │
│ MIA          │ SF             │ MIA               │ MIA           │ 2.05      │ 1.80           │ $5.25  │
│ DAL          │ TB             │ TB                │ DAL           │ 2.80      │ 1.46           │ $-5.00 │
│ GB           │ NO             │ GB                │ NO            │ 1.11      │ 7.25           │ $-5.00 │
└──────────────┴────────────────┴───────────────────┴───────────────┴───────────┴────────────────┴────────┘

       Resultados da Semana       
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Métrica              ┃ Valor   ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ Total de jogos       │ 16      │
│ Acertos              │ 7       │
│ Assertividade        │ 43.75%  │
│ Lucro total          │ $-26.06 │
│ Porcentagem de lucro │ -32.57% │
└──────────────────────┴─────────┘